In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.layers import GRU, SimpleRNN
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.models.keyedvectors as word2vec
import gc
from gensim.models import KeyedVectors
import gensim
import pandas as pd
import numpy as np

In [ ]:
list_classes = ['B-pers','I-pers', 'B-pro', 'I-pro','B-loc','I-loc','B-fac','I-fac','B-event','I-event','B-org','I-org','O']
list_columns = ['text','label']

In [ ]:
def label_to_one_hot(value):
  list_classes = ['B-pers','I-pers', 'B-pro', 'I-pro','B-loc','I-loc','B-fac','I-fac','B-event','I-event','B-org','I-org','O']
  if value == list_classes[0]:
    return [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[1]:
    return [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[2]:
    return [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[3]:
    return [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[4]:
    return [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[5]:
    return [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[6]:
    return [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
  if value == list_classes[7]:
    return [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
  if value == list_classes[8]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
  if value == list_classes[9]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
  if value == list_classes[10]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  if value == list_classes[11]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
  if value == list_classes[12]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
def read_data(filename):
    df= pd.DataFrame(columns=list_columns)
    
    f = open(filename, encoding="utf8")
    sentences = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')        
        sentence.append([splits[0], splits[-1]])

    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
        
    for sentence in sentences:
      text=''
      words=[]
      values=[]
      for item in sentence:
        words.append(item[0])
        values.append(item[1].replace('\n',''))
      _list=[]
      _list.append(' '.join(words))
      _list.append(' '.join(values))
      df2 = pd.DataFrame([_list], columns=list_columns)
      df=df.append(df2, ignore_index=True)
    return df

In [ ]:
y = train['label']
list_sentences_train = train['text']

In [ ]:
max_features = 250015
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)

y = [[label_to_one_hot(c) for c in ey.split(' ')] for ey in y]

In [ ]:
totalNumWords = [len(one_comment) for one_comment in list_tokenized_train]
plt.hist(totalNumWords,bins = np.arange(0,120,5))
plt.show()

In [ ]:
maxlen = 60
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y_t = pad_sequences(y, maxlen=maxlen)
print(len(X_t[0]))
print(len(y_t[0]))

In [ ]:
from keras.layers import Dense,Bidirectional Flatten
from keras.models import Sequential

In [ ]:
model2 = Sequential()
model2.add(Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False))
model2.add(Bidirectional(LSTM(30, return_sequences=True,name='lstm')))
model2.add(Bidirectional(LSTM(30, return_sequences=True,name='lstm')))
model2.add(Dense(13, activation='softmax'))

model2.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

In [ ]:
# Visualize the Model
plt.style.use("ggplot")
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.save('PER-NER-BILSTM.h5')